In [1]:
class Term:

    def _eq__from_Const(self,other):
        return False
    
    def _eq__from_Var(self,other):
        return False

    def _eq__from_Comb(self,other):
        return False

    def _eq__from_Abs(self,other):
        return False


In [2]:
class Const(Term):
    '''Constant'''

    def __init__(self,s):
        self.name=s

    def __hash__(self):
        return hash(self.name)

    def __eq__(self,other):
        return isinstance(other,Const) and \
               self.name == other.name

    def __str__(self):
        return self.name
    
    def frees(self):
        return set()

    def subst(self,f):
        return self
    
    def strip_comb(self):
        return [self]

In [3]:
assert Const('x').name == 'x'
assert len(Const('x').frees()) == 0
assert Const('x').subst({'x': Const('x')}).name == 'x'

In [4]:
def prime(v):
    return Var(v.name+"'")

def fresh(v,avoid):
    return fresh(prime(v),avoid) if v in avoid else v

In [5]:
class Var(Term):
    '''Variable'''

    def __init__(self,s):
        self.name=s

    def __hash__(self):
        return hash(self.name)

    def __eq__(self,other):
        return isinstance(other,Var) and self.name == other.name

    def __str__(self):
        return self.name

    def frees(self):
        return {self}

    def subst(self,f):
        return f.get(self.name,self)
    
    def strip_comb(self):
        return [self]

In [6]:
assert Var('x').name == 'x'
assert Var('x') == Var('x')
assert Var('x') != Var('y')
assert len(Var('x').frees()) == 1
assert Var('x').subst({'x': Const('y')}).name == 'y'
assert Var('x').subst({'y': Const('x')}).name == 'x'

In [7]:
class Comb(Term):
    '''Combination'''

    def __init__(self,f,x):
        self.rator = f
        self.rand = x

    def __hash__(self):
        return hash(self.rator) ^ hash(self.rand)

    def __eq__(self,other):
        return other._eq__from_Comb(self)
    
    def _eq__from_Comb(self,other_comb):
        return self.rand == other_comb.rand and \
               self.rator == other_comb.rator

    def __str__(self):
        #return ('({0.rator})'.format(self))
        #return ('(' + str(self.rator) + ' ' + str(self.rand) + ')')
        return '(' + ' '.join(map(str,self.strip_comb())) + ')'

    def frees(self):
        return (self.rator.frees() | self.rand.frees())

    def subst(self,f):
        return (Comb((self.rator).subst(f),(self.rand).subst(f)))
    
    def strip_comb(self):
        return self.rator.strip_comb() + [self.rand]

In [8]:
assert Comb(Var('x'),Const('y')) == Comb(Var('x'),Const('y'))
assert Comb(Var('x'),Const('y')) != Var('y')
assert Comb(Var('x'),Const('y')) != Comb(Var('x'),Var('y'))
assert Comb(Var('x'),Var('y')).rator.name == 'x'
assert Comb(Var('x'),Var('y')).strip_comb() == [Var('x'),Var('y')]
assert Comb(Var('x'),Comb(Var('x'),Var('y'))).strip_comb() == [Var('x'),Comb(Var('x'),Var('y'))]
assert Comb(Comb(Var('x'),Var('y')),Var('x')).strip_comb() == [Var('x'),Var('y'),Var('x')]
assert str(Comb(Comb(Var('x'),Var('y')),Var('z'))) == '(x y z)'
assert str(Comb(Var('x'),Comb(Var('y'),Var('z')))) == '(x (y z))'

In [9]:
class Abs(Term):

    def __init__(self,v,b):
        self.absvar = v
        self.body = b

    def __hash__(self):
        return hash(self.absvar) ^ hash(self.body)

    def __eq__(self,other):
        return isinstance(other,Abs) and self.absvar == other.absvar and self.body == other.body

    def __str__(self):
        return ('(\\'+str(self.absvar)+'. '+str(self.body)+')')

    def frees(self):
        fv = self.body.frees()
        return (fv - {self.absvar})

    def subst(self,f):
        v = self.absvar
        v1 = fresh(v,self.body.frees())
        f1 = f.copy()
        f1[v.name] = v1
        return (Abs(v1,self.body.subst(f1)))

    def strip_comb(self):
        return [self]

In [10]:
def showsubst(x,f):
    print(x,'|-->', x.subst(f))

In [11]:
showsubst(Var('x'),{'x': Const('c'), 'y': Var('x')})
showsubst(Comb(Const('d'),Var('x')),{'x': Const('c'), 'y': Var('x')})

x |--> c
(d x) |--> (d c)


In [16]:
xx = Var('x')
yy = Var('y')
cc = Const('c')
tm1 = Comb(xx,cc)
tm2 = Comb(tm1,yy)
tm3 = Comb(Abs(xx,tm2),tm2)
ff = {'x': yy, 'y': xx}
print(tm3)

((\x. (x c y)) (x c y))


In [13]:
tm3.subst(ff)

showsubst(tm3,ff)

((\x. (x c y)) (x c y)) |--> ((\x'. (x' c x)) (y c x))


In [14]:
print(tm3.frees())
print(Comb(tm3,tm3.subst(ff)).frees())

{<__main__.Var object at 0x104932320>, <__main__.Var object at 0x104932550>}
{<__main__.Var object at 0x104932320>, <__main__.Var object at 0x104932550>}
